In [7]:
import requests
import json
import re
from bs4 import BeautifulSoup

In [2]:
r = requests.get('http://www.siamensis.org/json?type=tree')

In [3]:
si_list = r.json()[0][0]

In [4]:
si_list.keys()

dict_keys(['data', 'attr', 'mlid', 'num_children', 'children'])

In [ ]:
si_list['children']

In [ ]:
numc = si_list['num_children']

In [ ]:
t = 'Genus: Oscillatoria <span class="num-children">(2)</span>'
ts = "Species : <em> Oscillatoria limosa</em>"

import re

In [ ]:
re.sub(r'\s:', ':', re.sub(r'\s*?\([0-9]+\)', '', re.sub(r'<[/\sa-zA-Z\"=-]+>\s?', '', t)))

In [133]:
def rmHTMLTag(string):
    # remove html tag first then remove number of children in parentheses
    return re.sub(r'\s+:', ':', re.sub(r'\s*?\([0-9]+\)', '', re.sub(r'<[/\sa-zA-Z\"=-]+>\s?', '', string)))

def printDataDown(jsonObj, higher_order, parent_id, my_id):
    prefix = '' + higher_order
    itemId = jsonObj['attr']['link'].split('/')[-1]
    tmp = rmHTMLTag(jsonObj['data'])
    # {0}: prefix space
    # {1}: data
    # {2}: old id
    # {3}: new id
    this_new_id = "{0}-{1}{2}".format(parent_id, tmp[0], my_id)
    this_new_id = re.sub(r'^-', '', this_new_id)
    print("{0}{1}, id {2}, new id {3}"
          .format(prefix, tmp, itemId, this_new_id))
    # print(getNodeData(int(itemId)))
    if 'num_children' in jsonObj:
        tmpId = 0
        for obj in jsonObj['children']:
            getDataDown(obj, ' '+prefix, this_new_id, tmpId)
            tmpId += 1
#     else:
#         itemId = jsonObj['attr']['link'].split('/')[-1]
#         tmp = rmHTMLTag(jsonObj['data'])
#         this_new_id = "{0}-{1}{2}".format(parent_id, tmp[0], my_id)
#         print("{0}{1}, id {2}, new id {3}"
#               .format(prefix, tmp, itemId, this_new_id))
#         print(getNodeData(int(itemId)))

In [134]:
printDataDown(si_list, '', '', '0')


Life taxonomy, id 673, new id L0
 Domain: Archaea, id 708, new id L0-D0
  Kingdom: Archaea, id 710, new id L0-D0-K0
   Phylum: Crenarchaeota, id 6183, new id L0-D0-K0-P0
   Phylum: Euryarchaeota, id 6184, new id L0-D0-K0-P1
 Domain: Bacteria, id 707, new id L0-D1
  Kingdom: Bacteria, id 711, new id L0-D1-K0
   Phylum: Acidobacteria, id 6210, new id L0-D1-K0-P0
   Phylum: Actinobacteria, id 6209, new id L0-D1-K0-P1
   Phylum: Aquificae, id 6208, new id L0-D1-K0-P2
   Phylum: Bacteroidetes, id 6207, new id L0-D1-K0-P3
   Phylum: Chlamydiae, id 6206, new id L0-D1-K0-P4
   Phylum: Chloroflexi, id 6205, new id L0-D1-K0-P5
   Phylum: Chrysiogenetes, id 6204, new id L0-D1-K0-P6
   Phylum: Cyanobacteria, id 6203, new id L0-D1-K0-P7
    Class: Cyanophyceae, id 41114, new id L0-D1-K0-P7-C0
     Order: Nostocales, id 41115, new id L0-D1-K0-P7-C0-O0
      Family: Oscillatoriaceae, id 41116, new id L0-D1-K0-P7-C0-O0-F0
       Genus: Oscillatoria, id 41117, new id L0-D1-K0-P7-C0-O0-F0-G0
        Spe

            Species: Chremistica guamauangensis, id 37354, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S3
            Species: Chremistica malayensis, id 37355, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S4
            Species: Chremistica moultoni, id 37356, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S5
            Species: Chremistica mussarens, id 37357, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S6
            Species: Chremistica nesiotes, id 37358, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S7
            Species: Chremistica numida, id 37359, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S8
            Species: Chremistica ochracea, id 37360, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S9
            Species: Chremistica pontianaka, id 37361, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S10
            Species: Chremistica siamensis, id 37362, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S11
            Species: Chremistica viridis, id 37363, new id L0-D2-K0-P0-C4-O2-S0-S0-F0-S0-T6-G1-S1

         Species: Macrobrachium malayanum (Roux, 1934), id 34629, new id L0-D2-K0-P0-C5-O1-I2-F1-G0-S14
         Species: Macrobrachium mieni Dang, 1975, id 34630, new id L0-D2-K0-P0-C5-O1-I2-F1-G0-S15
         Species: Macrobrachium mirabile (Kemp, 1917), id 34631, new id L0-D2-K0-P0-C5-O1-I2-F1-G0-S16
         Species: Macrobrachium neglectum (De Man, 1905), id 34632, new id L0-D2-K0-P0-C5-O1-I2-F1-G0-S17
         Species: Macrobrachium niphanae Shokita and Takeda, 1989, id 34633, new id L0-D2-K0-P0-C5-O1-I2-F1-G0-S18
         Species: Macrobrachium sintangense (De Man, 1898), id 34634, new id L0-D2-K0-P0-C5-O1-I2-F1-G0-S19
         Species: Macrobrachium sirindhorn Naiyanetr, 2001, id 34635, new id L0-D2-K0-P0-C5-O1-I2-F1-G0-S20
         Species: Macrobrachium thai Cai, Naiyanetr and Ng, 2004, id 34636, new id L0-D2-K0-P0-C5-O1-I2-F1-G0-S21
         Species: Macrobrachium tratense Cai, Naiyanetr and Ng, 2004, id 34637, new id L0-D2-K0-P0-C5-O1-I2-F1-G0-S22
         Species: Macrobra

          Species: Yasuhikotakia morleti , id 962, new id L0-D2-K0-P1-C5-S0-O2-F1-S4-G3-S5
          Species: Yasuhikotakia nigrolineata, id 965, new id L0-D2-K0-P1-C5-S0-O2-F1-S4-G3-S6
          Species: Yasuhikotakia sidthimunki , id 745, new id L0-D2-K0-P1-C5-S0-O2-F1-S4-G3-S7
          Species: Yasuhikotakia splendida, id 748, new id L0-D2-K0-P1-C5-S0-O2-F1-S4-G3-S8
       Family: Cyprinidae, id 990, new id L0-D2-K0-P1-C5-S0-O2-F2
        Subfamily: Alburninae, id 7196, new id L0-D2-K0-P1-C5-S0-O2-F2-S0
         Genus: Longiculter, id 7200, new id L0-D2-K0-P1-C5-S0-O2-F2-S0-G0
         Genus: Paralaubuca, id 7199, new id L0-D2-K0-P1-C5-S0-O2-F2-S0-G1
        Subfamily: Cyprininae - Cyprinini, id 4107, new id L0-D2-K0-P1-C5-S0-O2-F2-S1
         Genus: Catlocarpio, id 4108, new id L0-D2-K0-P1-C5-S0-O2-F2-S1-G0
          Species: Catlocarpio siamensis, id 4110, new id L0-D2-K0-P1-C5-S0-O2-F2-S1-G0-S0
         Genus: Neolissochilus, id 5065, new id L0-D2-K0-P1-C5-S0-O2-F2-S1-G1
       

         Species: Vanilla siamensis, id 8326, new id L0-D2-K3-D13-C0-O2-F0-S5-G11-S0
       Subfamily: Neottoidede, id 8013, new id L0-D2-K3-D13-C0-O2-F0-S6
        Genus: Nervilia, id 8316, new id L0-D2-K3-D13-C0-O2-F0-S6-G0
         Species: Nervilia aragoana, id 8395, new id L0-D2-K3-D13-C0-O2-F0-S6-G0-S0
         Species: Nervilia infundibulifolia, id 8317, new id L0-D2-K3-D13-C0-O2-F0-S6-G0-S1
         Species: Nervilia plicata, id 8318, new id L0-D2-K3-D13-C0-O2-F0-S6-G0-S2
       Subfamily: Orchidoideae, id 7802, new id L0-D2-K3-D13-C0-O2-F0-S7
        Genus: Habenaria, id 7803, new id L0-D2-K3-D13-C0-O2-F0-S7-G0
         Species: Habenaria dentata, id 34195, new id L0-D2-K3-D13-C0-O2-F0-S7-G0-S0
         Species: Habenaria lindleyana, id 7804, new id L0-D2-K3-D13-C0-O2-F0-S7-G0-S1
         Species: Habenaria rhodocheila, id 8076, new id L0-D2-K3-D13-C0-O2-F0-S7-G0-S2
       Subfamily: Vandoideae, id 8012, new id L0-D2-K3-D13-C0-O2-F0-S8
        Genus: Eulophia, id 8459, new id 

In [37]:
cont = requests.get('http://www.siamensis.org/species_index/node/709')

In [38]:
soup = BeautifulSoup(cont.text, 'html.parser')

print(soup.prettify())
# print(soup.get_text())

for tag in soup.find_all(True):
#     print(tag.name)
#     if tag.has_attr('class'):
#         print(tag)
    if tag.has_attr('src'):
        print(tag.get('src'))

<div class="node-body">
 <h2 class="node-title">
  Life taxonomy
 </h2>
 <div class="node-header">
 </div>
 <p>
  Life taxonomy ของเรา อ้างอิงตาม "Catalogue of Life" (
  <a href="http://www.catalogueoflife.org/annual-checklist/2010/browse/tree">
   http://www.catalogueoflife.org/annual-checklist/2010/browse/tree
  </a>
  ) โดยมี 7 อาณาจักร คือ Archaea, Bacteria, Animalia, Chromista, Fungi, Plantae และ Protozoa ใน 3 โดเมน (domain)
  <br/>
  <br/>
  วิธีการใช้ ดัชนีสิ่งมีชีวิต มีสองรูปแบบ
  <br/>
  1. คือกดที่ Folder ทางด้านซ้ายมือไปเรื่อยๆ จนถึงสิ่งมีชีวิตที่คุณต้องการ หรือ
  <br/>
  2. ใส่ชื่อ หรือ คำใดๆที่คุณต้องการค้นหา จะเป็น ชื่อไทย ชื่อสามัญ หรือ ชื่อวิทยาศาสตร์ หรือ ข้อความเช่น เฉพาะถิ่นของไทย สัตว์คุ้มครอง ป่าพรุ หรือ ภาคใต้ สิ่งมีชีวิตที่มีคำเหล่านี้อยู่ในเนื้อความก็จะปรากฏขึ้นในกล่องทางด้านขวา (ที่ท่านกำลังอ่านอยู่นี้)
  <br/>
  <br/>
  แต่ถ้าหากคุณมีภาพสิ่งมีชีวิต แต่ไม่แน่ใจหรือไม่ทราบว่าชื่ออะไร นำภาพไปแปะถามที่ บอร์ดพูดคุย จะง่ายกว่าครับ
 </p>
 <div class="node-submitted">

[<p>Life taxonomy ของเรา อ้างอิงตาม "Catalogue of Life" ( <a href="http://www.catalogueoflife.org/annual-checklist/2010/browse/tree">http://www.catalogueoflife.org/annual-checklist/2010/browse/tree</a>) โดยมี 7 อาณาจักร คือ Archaea, Bacteria, Animalia, Chromista, Fungi, Plantae และ Protozoa ใน 3 โดเมน (domain)<br/><br/>วิธีการใช้ ดัชนีสิ่งมีชีวิต มีสองรูปแบบ<br/>1. คือกดที่ Folder ทางด้านซ้ายมือไปเรื่อยๆ จนถึงสิ่งมีชีวิตที่คุณต้องการ หรือ<br/>2. ใส่ชื่อ หรือ คำใดๆที่คุณต้องการค้นหา จะเป็น ชื่อไทย ชื่อสามัญ หรือ ชื่อวิทยาศาสตร์ หรือ ข้อความเช่น เฉพาะถิ่นของไทย สัตว์คุ้มครอง ป่าพรุ หรือ ภาคใต้ สิ่งมีชีวิตที่มีคำเหล่านี้อยู่ในเนื้อความก็จะปรากฏขึ้นในกล่องทางด้านขวา (ที่ท่านกำลังอ่านอยู่นี้) <br/><br/>แต่ถ้าหากคุณมีภาพสิ่งมีชีวิต แต่ไม่แน่ใจหรือไม่ทราบว่าชื่ออะไร นำภาพไปแปะถามที่ บอร์ดพูดคุย จะง่ายกว่าครับ</p>,
 <p class="child-help">*<u>หมายเหตุ</u> เมื่อกดที่รูปภาพ จะไปยังหน้าดัชนีนั้นๆทันที</p>]

In [80]:
def getNodeData(node_id):
    # make request
    cont = requests.get('http://www.siamensis.org/species_index/node/{0}'.format(node_id))
    soup = BeautifulSoup(cont.text, 'html.parser')

    # retrieving data
    title = soup.h2.text
    header_img = None if soup.find_all('img', 'imagecache-jstree_header') == [] else soup.find_all('img', 'imagecache-jstree_header')[0]['src']
    description = soup.find_all('p')
    all_imgs = [img.get('src') for img in soup.find_all(True) 
                if img.has_attr('src') and 'jstree_header' not in img.get('src')]
    author = [re.sub(r'Authenticated user |\s+$', '', i.get_text()) for i in soup.find_all('div') 
              if i.has_attr('class') and 'node-submitted' in i.get('class')][0]
    editors = [re.sub(r'\s+', ' ', i.get_text()) for i in soup.ul.find_all('li')]

    # pack in dict
    data = {
        'title': title,
        'header_img': header_img,
        'description': description,
        'all_imgs': all_imgs,
        'author': author,
        'editors': editors,
    }

    return data

In [70]:
getNodeData('673')

{'all_imgs': ['http://www.siamensis.org/sites/default/files/imagecache/specie_child_thumb/staphylococcus_aureus.jpg',
  'http://www.siamensis.org/sites/default/files/imagecache/specie_child_thumb/archaea_inter-species_interaction.png',
  'http://www.siamensis.org/sites/default/files/imagecache/specie_child_thumb/axis_2.jpg',
  'http://www.siamensis.org/sites/default/files/imagecache/specie_child_thumb/250px-paramecium.jpg',
  'http://www.siamensis.org/sites/default/files/imagecache/specie_child_thumb/kingdom_600.jpg'],
 'author': 'เขียนโดย admin เมื่อ  7 September 10 12:04',
 'description': 'Life taxonomy ของเรา อ้างอิงตาม "Catalogue of Life" ( http://www.catalogueoflife.org/annual-checklist/2010/browse/tree) โดยมี 7 อาณาจักร คือ Archaea, Bacteria, Animalia, Chromista, Fungi, Plantae และ Protozoa ใน 3 โดเมน (domain)วิธีการใช้ ดัชนีสิ่งมีชีวิต มีสองรูปแบบ1. คือกดที่ Folder ทางด้านซ้ายมือไปเรื่อยๆ จนถึงสิ่งมีชีวิตที่คุณต้องการ หรือ2. ใส่ชื่อ หรือ คำใดๆที่คุณต้องการค้นหา จะเป็น ชื่อไทย ชื